In [1]:
import re
import pandas as pd
from wordcloud import WordCloud
from nltk.tokenize import RegexpTokenizer
import spacy
from spacy.lang.id import Indonesian
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [2]:
nlp = spacy.blank("id")
nlp.add_pipe("lemmatizer", config={"mode": "lookup"})
nlp.initialize()

In [3]:
class TextPreprocessing:
    def __init__(self):
        # Initialize instances
        self.tokenizer = RegexpTokenizer(r'\w+')
        self.stopwords_factory = StopWordRemoverFactory()
        self.stemmer_factory = StemmerFactory()
        
        # initialize additional stopwords and remove 'tahu' from stopwords list
        self.additional_stopwords = ['pasal', 'ayat', 'huruf', 'nomor', 'diatur', 'atur', 'suai', 'puluh', 'persen', 'tahun', 'perundang', 'undang', 'direktorat', 'direktur', 'menteri', 'presiden', 'republik', 'indonesia']
        self.stopwords = self.stopwords_factory.get_stop_words() + self.additional_stopwords

    def tokenize(self, list_content):
        '''Tokenize sentences into words from list of sentences'''
        return [self.tokenizer.tokenize(i.lower()) for i in list_content]


    def remove_stopwords(self, list_content):
        '''Remove stopwords from list of words in list of contents'''
        return [[j for j in i if j not in self.stopwords] for i in list_content]


    def remove_non_alpha(self, list_content):
        '''Remove non-alphabetic characters from list of words in list of contents'''
        return [[j for j in i if j.isalpha()] for i in list_content]

    
    def lemmatize_words(self, list_content):
        '''Lemmatize words in list of words in list of contents'''
        lemmatized_content = []
        for sublist in list_content:
            lemmatized_sublist = []
            for word in sublist:
                lemmatized_sublist.extend(token.lemma_ for token in nlp(word))
            lemmatized_content.append(lemmatized_sublist)
        return lemmatized_content
    

    def stem_words(self, list_content):
        '''Stem words in list of words in list of contents'''
        stemmer = self.stemmer_factory.create_stemmer()
        return [[stemmer.stem(j) for j in i] for i in list_content]
    

    def list1d_to_string(self, list_content):
        '''Convert list of words to string'''
        return str(list_content).strip('[]')


    def clean_doc(self, list_content):
        '''convert list of list of words to string'''
        return ' '.join([item for sublist in list_content for item in sublist])

In [5]:
# read docs_df
docs_df = pd.read_csv('../../data/law_analyzer_new4_title-content.csv')
docs_df

,_id,title,content
0,221-pmk.010-2015,PENGENAAN BEA MASUK ANTI DUMPING TERHADAP IMPO...,Terhadapimporproduk _Biaxially_ _Oriented_ _P...
1,pp-21-tahun-2005,STANDAR NASIONAL PENDIDIKAN,KETENTUAN UMUM Dalam Peraturan Pemerintah ini ...
2,pp-51-tahun-2008,PERUBAHAN NAMA KABUPATEN YAPEN WAROPEN MENJADI...,Nama Kabupaten Yapen Waropen sebagai daerah o...
3,62-pmk.04-2018,PERUBAHAN KEDUA ATAS PERATURAN MENTERI KEUANGA...,(1) Metode pengulangan ( _fallback_) sebagaim...
4,42-pmk.05-2017,PERUBAHAN ATAS PERATURAN MENTERI KEUANGAN NOMO...,(1) Laporan Keuangan BLU sebagaimana dimaksud...
...,...,...,...
2997,pp-9-tahun-2014,PELAKSANAAN UNDANG-UNDANG NOMOR 4 TAHUN 2011 T...,KETENTUAN UMUM Dalam Peraturan Pemerintah ini ...
2998,pp-25-tahun-2017,PEMBERIAN TUNJANGAN HARI RAYA DALAM TAHUN ANGG...,Dalam Peraturan Pemerintah ini yang dimaksud ...
2999,160-pmk.04-2010,PELAKSANAAN PENYALURAN DANA ALOKASI KHUSUS TAH...,(1)PenyaluranDanaAlokasiKhusus(DAK)TahunAngga...
3000,146-pmk.05-2019,TARIF LAYANAN BADAN LAYANAN UMUM POLITEKNIK KE...,Tarif layanan Badan Layanan Umum Politeknik K...


In [6]:
text_preprocessing = TextPreprocessing()

# title
docs_df['clean_title'] = docs_df['title'].apply(lambda x: text_preprocessing.tokenize([x]))
docs_df['clean_title'] = docs_df['clean_title'].apply(lambda x: text_preprocessing.remove_non_alpha(x))
docs_df['clean_title'] = docs_df['clean_title'].apply(lambda x: text_preprocessing.lemmatize_words(x))
docs_df['clean_title'] = docs_df['clean_title'].apply(lambda x: text_preprocessing.remove_stopwords(x))
docs_df['clean_title'] = docs_df['clean_title'].apply(lambda x: text_preprocessing.clean_doc(x))

# content
docs_df['clean_content'] = docs_df['content'].apply(lambda x: text_preprocessing.tokenize([x]))
docs_df['clean_content'] = docs_df['clean_content'].apply(lambda x: text_preprocessing.remove_non_alpha(x))
docs_df['clean_content'] = docs_df['clean_content'].apply(lambda x: text_preprocessing.lemmatize_words(x))
docs_df['clean_content'] = docs_df['clean_content'].apply(lambda x: text_preprocessing.remove_stopwords(x))
docs_df['clean_content'] = docs_df['clean_content'].apply(lambda x: text_preprocessing.clean_doc(x))

docs_df

,_id,title,content,clean_title,clean_content
0,221-pmk.010-2015,PENGENAAN BEA MASUK ANTI DUMPING TERHADAP IMPO...,Terhadapimporproduk _Biaxially_ _Oriented_ _P...,bea anti dumping impor produk oriented polyeth...,terhadapimporproduk negara india rakyat tiongk...
1,pp-21-tahun-2005,STANDAR NASIONAL PENDIDIKAN,KETENTUAN UMUM Dalam Peraturan Pemerintah ini ...,standar nasional didik,umum perintah standar nasional didik kriteria ...
2,pp-51-tahun-2008,PERUBAHAN NAMA KABUPATEN YAPEN WAROPEN MENJADI...,Nama Kabupaten Yapen Waropen sebagai daerah o...,ubah nama kabupaten yapen waropen kabupaten pu...,nama kabupaten yapen waropen daerah otonom wil...
3,62-pmk.04-2018,PERUBAHAN KEDUA ATAS PERATURAN MENTERI KEUANGA...,(1) Metode pengulangan ( _fallback_) sebagaim...,ubah uang pmk nilai pabean hitung bea,metode ulang ulang prinsip tata wajar konsiste...
4,42-pmk.05-2017,PERUBAHAN ATAS PERATURAN MENTERI KEUANGAN NOMO...,(1) Laporan Keuangan BLU sebagaimana dimaksud...,ubah uang pmk sistem akuntansi lapor uang bada...,lapor uang blu rangka pengintegrasian lapor ua...
...,...,...,...,...,...
2997,pp-9-tahun-2014,PELAKSANAAN UNDANG-UNDANG NOMOR 4 TAHUN 2011 T...,KETENTUAN UMUM Dalam Peraturan Pemerintah ini ...,laksana informasi geospasial,umum perintah geospasial sifat keruangan posis...
2998,pp-25-tahun-2017,PEMBERIAN TUNJANGAN HARI RAYA DALAM TAHUN ANGG...,Dalam Peraturan Pemerintah ini yang dimaksud ...,tunjang raya anggar gawai negeri sipil prajuri...,perintah gawai negeri sipil disingkat pns warg...
2999,160-pmk.04-2010,PELAKSANAAN PENYALURAN DANA ALOKASI KHUSUS TAH...,(1)PenyaluranDanaAlokasiKhusus(DAK)TahunAngga...,laksana salur dana alokasi anggar,penyalurandanaalokasikhusus dak tahap ii tahap...
3000,146-pmk.05-2019,TARIF LAYANAN BADAN LAYANAN UMUM POLITEKNIK KE...,Tarif layanan Badan Layanan Umum Politeknik K...,tarif layan badan layan umum politeknik sehat ...,tarif layan badan layan umum politeknik sehat ...


In [8]:
# save to csv
docs_df.to_csv('data/law_analyzer_new4_title-content-clean.csv', index=False)